Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# Set Cover problem

See: https://en.wikipedia.org/wiki/Set_cover_problem

In [1]:
# !pip install icecream
from random import random, seed
from itertools import product
import numpy as np
from icecream import ic

## Reproducible Initialization

If you want to get reproducible results, use `rng` (and restart the kernel); for non-reproducible ones, use `np.random`.

In [2]:
UNIVERSE_SIZE = 100_000
NUM_SETS = 10_000
DENSITY = 0.3



data = [
    [100, 10, 0.2],
    [1000, 100, 0.2],
    [10000, 1000, 0.3],
    [100000, 10000, 0.1],
    [100000, 10000, 0.2],
    [100000, 10000, 0.3]
]

rng = np.random.Generator(np.random.PCG64([UNIVERSE_SIZE, NUM_SETS, int(10_000 * DENSITY)]))

## Helper Functions

In [3]:
def valid(solution):
    """Checks wether solution is valid (ie. covers all universe)"""
    return np.all(np.logical_or.reduce(SETS[solution]))


def cost(solution):
    """Returns the cost of a solution (to be minimized)"""
    return COSTS[solution].sum()

## Have Fun!

In [5]:

# Apply the greedy algorithm to the first set of LIST_OF_SETS
class TabuSearch:
    def __init__(self, list_of_sets, max_iterations, tabu_tenure, max_no_improve):
        self.list_of_sets = list_of_sets
        self.max_iterations = max_iterations
        self.tabu_tenure = tabu_tenure
        self.max_no_improve = max_no_improve
        self.tabu_list = []
        self.best_solution = None
        self.best_cost = float('inf')
        self.evaluations=0

    def _get_coverage(self, selected_sets):
        # Calculate the coverage of the selected sets
        sets, _ = self.list_of_sets
        covered = np.any(sets[selected_sets], axis=0)
        return covered

    def _evaluate_solution(self, selected_sets):
        # Evaluate the solution based on the total cost of the selected sets
        self.evaluations+=1
        sets, costs = self.list_of_sets
        total_cost = np.sum(costs[selected_sets])
        covered_elements = self._get_coverage(selected_sets)
        num_uncovered = np.sum(~covered_elements)
        return total_cost, num_uncovered

    def _generate_initial_solution(self):
        # Greedy initialization: select sets that cover the most elements
        sets, _ = self.list_of_sets
        num_elements = sets.shape[1]
        uncovered_elements = np.ones(num_elements, dtype=bool)
        selected_sets = []

        while np.any(uncovered_elements):
            # Select the set that covers the most uncovered elements
            cover_count = np.sum(sets[:, uncovered_elements], axis=1)
            best_set = np.argmax(cover_count)
            selected_sets.append(best_set)
            uncovered_elements = uncovered_elements & ~sets[best_set]

        return selected_sets

    def _get_neighborhood(self, current_solution):
        # Generate neighborhood by adding or removing one set from the current solution
        sets, _ = self.list_of_sets
        num_sets = sets.shape[0]
        neighborhood = []

        # Try adding a new set
        for s in range(num_sets):
            if s not in current_solution:
                new_solution = current_solution + [s]
                neighborhood.append(new_solution)

        # Try removing a set
        for s in current_solution:
            new_solution = [i for i in current_solution if i != s]
            neighborhood.append(new_solution)

        return neighborhood

    def run(self):
        current_solution = self._generate_initial_solution()
        current_cost, num_uncovered = self._evaluate_solution(current_solution)
        best_solution = current_solution
        best_cost = current_cost
        no_improve_count = 0

        for iteration in range(self.max_iterations):
            # print(f"Iteration {iteration + 1}/{self.max_iterations}")
            neighborhood = self._get_neighborhood(current_solution)
            best_neigh_solution = None
            best_neigh_cost = float('inf')

            # Evaluate all neighbors
            for neighbor in neighborhood:
                if neighbor not in self.tabu_list:
                    neigh_cost, neigh_uncovered = self._evaluate_solution(neighbor)

                    # Only consider valid solutions that cover all elements
                    if neigh_uncovered == 0 and neigh_cost < best_neigh_cost:
                        best_neigh_solution = neighbor
                        best_neigh_cost = neigh_cost

            # If no valid neighbor found, stop
            if best_neigh_solution is None:
                break

            # Update current solution to best neighbor
            current_solution = best_neigh_solution
            current_cost = best_neigh_cost

            # Update tabu list
            self.tabu_list.append(current_solution)
            if len(self.tabu_list) > self.tabu_tenure:
                self.tabu_list.pop(0)

            # Update best solution if necessary
            if current_cost < best_cost:
                best_solution = current_solution
                best_cost = current_cost
                no_improve_count = 0
            else:
                no_improve_count += 1

            # Stop if no improvement for too long
            if no_improve_count >= self.max_no_improve:
                break

        self.best_solution = best_solution
        self.best_cost = best_cost
        return best_solution, best_cost, self.evaluations









In [10]:

# Apply the greedy algorithm to the first set of LIST_OF_SETS
class TabuSearch:
    def __init__(self, list_of_sets, max_iterations, tabu_tenure, max_no_improve):
        self.list_of_sets = list_of_sets
        self.max_iterations = max_iterations
        self.tabu_tenure = tabu_tenure
        self.max_no_improve = max_no_improve
        self.tabu_list = []
        self.best_solution = None
        self.best_cost = float('inf')
        self.evaluations=0

    def _get_coverage(self, selected_sets):
        # Calculate the coverage of the selected sets
        sets, _ = self.list_of_sets
        covered = np.any(sets[selected_sets], axis=0)
        return covered

    def _evaluate_solution(self, selected_sets):
        # Evaluate the solution based on the total cost of the selected sets
        covered_elements = self._get_coverage(selected_sets)
        num_uncovered = np.sum(~covered_elements)
        return cost(selected_sets), num_uncovered

    def _generate_initial_solution(self):
        # Greedy initialization: select sets that cover the most elements
        sets, _ = self.list_of_sets
        num_elements = sets.shape[1]
        uncovered_elements = np.ones(num_elements, dtype=bool)
        selected_sets = []

        while np.any(uncovered_elements):
            # Select the set that covers the most uncovered elements
            cover_count = np.sum(sets[:, uncovered_elements], axis=1)
            best_set = np.argmax(cover_count)
            selected_sets.append(best_set)
            uncovered_elements = uncovered_elements & ~sets[best_set]

        return selected_sets

    def _get_neighborhood(self, current_solution):
        # Generate neighborhood by adding or removing one set from the current solution
        sets, _ = self.list_of_sets
        num_sets = sets.shape[0]
        neighborhood = []

        # Try adding a new set
        for s in range(num_sets):
            if s not in current_solution:
                new_solution = current_solution + [s]
                neighborhood.append(new_solution)

        # Try removing a set
        for s in current_solution:
            new_solution = [i for i in current_solution if i != s]
            neighborhood.append(new_solution)

        return neighborhood

    def run(self):
        current_solution = self._generate_initial_solution()
        current_cost, num_uncovered = self._evaluate_solution(current_solution)
        best_solution = current_solution
        best_cost = current_cost
        no_improve_count = 0

        for iteration in range(self.max_iterations):
            print(f"Iteration {iteration + 1}/{self.max_iterations}")
            neighborhood = self._get_neighborhood(current_solution)
            best_neigh_solution = None
            best_neigh_cost = float('inf')

            # Evaluate all neighbors
            for neighbor in neighborhood:
                if neighbor not in self.tabu_list:
                    neigh_cost, neigh_uncovered = self._evaluate_solution(neighbor)

                    # Only consider valid solutions that cover all elements
                    if neigh_uncovered == 0 and neigh_cost < best_neigh_cost:
                        best_neigh_solution = neighbor
                        best_neigh_cost = neigh_cost

            # If no valid neighbor found, stop
            if best_neigh_solution is None:
                break

            # Update current solution to best neighbor
            current_solution = best_neigh_solution
            current_cost = best_neigh_cost

            # Update tabu list
            self.tabu_list.append(current_solution)
            if len(self.tabu_list) > self.tabu_tenure:
                self.tabu_list.pop(0)

            # Update best solution if necessary
            if current_cost < best_cost:
                best_solution = current_solution
                best_cost = current_cost
                no_improve_count = 0
            else:
                no_improve_count += 1

            # Stop if no improvement for too long
            if no_improve_count >= self.max_no_improve:
                break

        self.best_solution = best_solution
        self.best_cost = best_cost
        return best_solution, best_cost, self.evaluations


In [ ]:


max_iterations = 1000
tabu_tenure = 50
max_no_improve = 100

for mar in (data):
    SETS = np.random.random((mar[1], mar[0])) < mar[2]
    for s in range(mar[0]):
        if not np.any(SETS[:, s]):
            SETS[np.random.randint(mar[1]), s] = True
    COSTS = np.power(SETS.sum(axis=1), 1.1)
    
    tabu_search = TabuSearch((SETS,COSTS), max_iterations, tabu_tenure, max_no_improve)
    best_solution, best_cost, num_eval = tabu_search.run()
    ic(valid(best_solution), cost(best_solution), num_eval, len(best_solution))
    
    

ic| valid(best_solution): True
    cost(best_solution): 277.7703113168639
    num_eval: 0
    len(best_solution): 9


Iteration 1/1000
Iteration 2/1000
Iteration 3/1000
Iteration 1/1000
Iteration 2/1000
Iteration 3/1000
Iteration 4/1000
Iteration 5/1000
Iteration 6/1000
Iteration 7/1000
Iteration 8/1000
Iteration 9/1000
Iteration 10/1000
Iteration 11/1000
Iteration 12/1000
Iteration 13/1000
Iteration 14/1000
Iteration 15/1000
Iteration 16/1000
Iteration 17/1000
Iteration 18/1000
Iteration 19/1000
Iteration 20/1000
Iteration 21/1000
Iteration 22/1000
Iteration 23/1000
Iteration 24/1000
Iteration 25/1000
Iteration 26/1000
Iteration 27/1000
Iteration 28/1000
Iteration 29/1000
Iteration 30/1000
Iteration 31/1000
Iteration 32/1000
Iteration 33/1000
Iteration 34/1000
Iteration 35/1000
Iteration 36/1000
Iteration 37/1000
Iteration 38/1000
Iteration 39/1000
Iteration 40/1000
Iteration 41/1000
Iteration 42/1000
Iteration 43/1000
Iteration 44/1000
Iteration 45/1000
Iteration 46/1000


ic| valid(best_solution): True
    cost(best_solution): 5635.54941312594
    num_eval: 0
    len(best_solution): 16


Iteration 47/1000
Iteration 48/1000
Iteration 49/1000
Iteration 50/1000
Iteration 51/1000
Iteration 52/1000
Iteration 53/1000
Iteration 54/1000
Iteration 55/1000
Iteration 56/1000
Iteration 57/1000
Iteration 58/1000
Iteration 59/1000
Iteration 60/1000
Iteration 61/1000
Iteration 62/1000
Iteration 63/1000
Iteration 64/1000
Iteration 65/1000
Iteration 66/1000
Iteration 67/1000
Iteration 68/1000
Iteration 69/1000
Iteration 70/1000
Iteration 71/1000
Iteration 72/1000
Iteration 73/1000
Iteration 74/1000
Iteration 75/1000
Iteration 76/1000
Iteration 77/1000
Iteration 78/1000
Iteration 79/1000
Iteration 80/1000
Iteration 81/1000
Iteration 82/1000
Iteration 83/1000
Iteration 84/1000
Iteration 85/1000
Iteration 86/1000
Iteration 87/1000
Iteration 88/1000
Iteration 89/1000
Iteration 90/1000
Iteration 91/1000
Iteration 92/1000
Iteration 93/1000
Iteration 94/1000
Iteration 95/1000
Iteration 96/1000
Iteration 97/1000
Iteration 98/1000
Iteration 99/1000
Iteration 100/1000
Iteration 1/1000
Iteration 

ic| valid(best_solution): True
    cost(best_solution): 107996.85591312486
    num_eval: 0
    len(best_solution): 16


Iteration 99/1000
Iteration 100/1000
